<a href="https://colab.research.google.com/github/jokedetone/Scrapper_public_data_PY/blob/main/PYTHON_DATALCH_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FUNCIONA CON XPATH**

Permite descargar los ZIPS de [**PGN PARAGUAY**](https://datos.hacienda.gov.py/data/pgn-gasto/descargas ).

### **Realizado:**
*   Descarga y organiza los ZIPS en carpetas individuales por año

*   Extraer cada uno de los ZIPS dentro de sus respectivas carpetas y eliminar los ZIPS una vez que se cuente con el CSV.

### **Falta:**
*   Limpiar el dataset final

*   Análisis exploratorio



In [ ]:
!apt-get update
!apt install -y chromium-chromedriver
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os
import time
import requests
import zipfile
from glob import glob
import pandas as pd
from pprint import pprint

# Set up Selenium in Colab
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# URL to scrape
url = "https://datos.hacienda.gov.py/data/pgn-gasto/descargas"

# Years we want to download and their respective folders
target_years = {
    '2019': '2019_data',
    '2020': '2020_data',
    '2021': '2021_data',
    '2022': '2022_data',
    '2023': '2023_data',
    '2024': '2024_data',
    '2025': '2025_data'
}

def setup_folders():
    """Create folders for each year if they don't exist"""
    for folder in target_years.values():
        os.makedirs(folder, exist_ok=True)
    print("Created year-specific folders")

def extract_zip(zip_path, extract_to):
    """Extract a ZIP file to specified directory"""
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"  ✓ Extracted to {extract_to}")
        return True
    except zipfile.BadZipFile:
        print(f"  ✗ Error: Bad ZIP file - {zip_path}")
        return False
    except Exception as e:
        print(f"  ✗ Extraction failed: {str(e)}")
        return False



try:
    # Create folders first
    setup_folders()

    # Load the page
    print("Loading page...")
    driver.get(url)
    time.sleep(5)  # Wait for JavaScript to load completely

    # Find all download links using XPath
    download_links = driver.find_elements(
        By.XPATH, "//a[contains(translate(@href, 'ZIP', 'zip'), 'zip')]"
    )

    if not download_links:
        print("No ZIP download links found!")
    else:
        print(f"Found {len(download_links)} ZIP files. Filtering for target years...")

        downloaded_files = {year: 0 for year in target_years}
        extracted_files = {year: 0 for year in target_years}

        for link in download_links:
            try:
                # Get the download URL
                zip_url = link.get_attribute('href')
                filename = zip_url.split('/')[-1]

                # Determine which year this file belongs to
                file_year = None
                for year in target_years:
                    if year in filename:
                        file_year = year
                        break

                if file_year:
                    # Handle relative URLs
                    if not zip_url.startswith('http'):
                        zip_url = f"https://datos.hacienda.gov.py{zip_url}" if zip_url.startswith('/') else f"{url}/{zip_url}"

                    folder_path = target_years[file_year]
                    zip_path = os.path.join(folder_path, filename)

                    print(f"\nProcessing {filename}:")
                    print(f"- Downloading to {folder_path}...")

                    # Download the file
                    with requests.get(zip_url, stream=True) as r:
                        r.raise_for_status()
                        with open(zip_path, 'wb') as f:
                            for chunk in r.iter_content(chunk_size=8192):
                                if chunk:
                                    f.write(chunk)

                    downloaded_files[file_year] += 1
                    print(f"  ✓ Download complete")

                    # Extract the ZIP file
                    print(f"- Extracting contents...")
                    if extract_zip(zip_path, folder_path):
                        extracted_files[file_year] += 1

                    # Optionally remove the ZIP after extraction (uncomment to enable)
                    os.remove(zip_path)
                    print("  ✓ Removed ZIP file after extraction")

            except Exception as e:
                print(f"✗ Failed to process {filename}: {str(e)}")

        # Print download summary
        print("\n=== Download Summary ===")
        for year in target_years:
            print(f"{year}:")
            print(f"  - Downloaded: {downloaded_files[year]} files")
            print(f"  - Extracted: {extracted_files[year]} files")

        total_downloaded = sum(downloaded_files.values())
        total_extracted = sum(extracted_files.values())
        print(f"\nTotal: {total_downloaded} files downloaded, {total_extracted} files extracted")

finally:
    driver.quit()
    print("\nProcess completed. You can find the files in their respective year folders.")



In [ ]:
#filtrar descripcionEntidad 005-MINISTERIO DE DEFENSA NACIONAL

def load_year_data(folder_path, year):
    """Load all CSV files from a year folder into a DataFrame,
    combine them, filter by 'descripcionEntidad', and remove individual CSVs."""
    csv_files = glob(os.path.join(folder_path, "*.csv"))
    if not csv_files:
        print(f"No CSV files found in {folder_path}")
        return None

    print(f"\nLoading {len(csv_files)} CSV files for {year}:")

    year_dfs = []
    for csv_file in csv_files:
        try:
            df = pd.read_csv(csv_file, encoding='ISO-8859-1', sep=',')
            df['Year'] = year  # Add year column
            year_dfs.append(df)
            print(f"  ✓ Loaded {os.path.basename(csv_file)} ({len(df)} rows)")
        except Exception as e:
            print(f"  ✗ Failed to load {csv_file}: {str(e)}")

    if year_dfs:
        combined_df = pd.concat(year_dfs, ignore_index=True)

        # Filter the combined DataFrame
        filtered_df = combined_df[combined_df['descripcionEntidad'] == '005-MINISTERIO DE DEFENSA NACIONAL']

        combined_csv_path = os.path.join(folder_path, f"combined_{year}.csv")
        filtered_df.to_csv(combined_csv_path, index=False)  # Save filtered DataFrame
        print(f"Saved filtered data to '{combined_csv_path}'")

        # Remove individual CSVs
        for csv_file in csv_files:
            os.remove(csv_file)
            print(f"Removed: {csv_file}")

        return filtered_df  # Return the filtered DataFrame
    return None

# Load and combine data
print("\nProcessing CSV files...")
yearly_dfs = {}  # Dictionary to store DataFrames for each year

for year, folder in target_years.items():
    year_df = load_year_data(folder, year)
    if year_df is not None:
        yearly_dfs[year] = year_df
        print(f"{year} DataFrame: {len(year_df)} rows")

# Combine all DataFrames
if yearly_dfs:
    combined_df = pd.concat(yearly_dfs.values(), ignore_index=True)
    print(f"\nFinal Combined DataFrame: {len(combined_df)} rows")

    # Save to CSV
    combined_df.to_csv('PRESU_2019-2025_COMBI.csv', index=False)
    print("\nSaved combined data to 'PRESU_2019-2025_COMBI.csv'")
else:
    print("No data was loaded from any year")

# VERSION FINAL DE LA EXTRACCIÓN DE COTIZACION
#### DESCRIPCIÓN

*   Realiza la extracción de los datos desde el año 2019 hasta el 2025
*   La tabla de cotizaciones desde el mes de Enero de 2019 hasta el 16-05-2019 no cuenta con el tfoot en el que se coloca el cierre del día para la compra como para la venta. A partir del siguiente día hábil, se cuenta con esa información directamente en el tfoot de la tabla.






In [ ]:
# Script para extraer datos de cotizaciones del Banco Central del Paraguay
# URL: https://www.bcp.gov.py/webapps/web/cotizacion/referencial-fluctuante

# Instalar las dependencias necesarias
!pip install selenium
!pip install beautifulsoup4
!pip install pandas
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin/


In [ ]:
# Import required libraries
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import os
from datetime import datetime
# Configure Chrome options for Colab
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Access the webpage
url = "https://www.bcp.gov.py/webapps/web/cotizacion/referencial-fluctuante"
driver.get(url)
print(f"Accessing URL: {url}")

# Wait for the page to load
wait = WebDriverWait(driver, 10)
print("Waiting for the page to load...")
try:
    # Wait for the calendar UI to appear
    wait.until(
        EC.presence_of_element_located((By.XPATH, "//div[@id='ui-datepicker-div']"))
    )
    print("Datepicker UI detected successfully.")
except Exception as e:
    print(f"Failed to detect the datepicker: {str(e)}")
    driver.quit()
    exit()
# Hardcoded start year
start_year = 2019

# Get the current year dynamically
current_year = datetime.now().year

# Get month names in Spanish for filenames
month_names = [
    "Ene", "Feb", "Mar", "Abr", "May", "Jun",
    "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"
]

# Define the cutoff date
cutoff_date = datetime(2019, 5, 16)

# Loop through all years from start_year to current_year
for year in range(start_year, current_year + 1):  # Include the current year
    print(f"Processing year: {year}")
    for month_index in range(12):  # Loop through all months (0 to 11)
        try:
            # Open the calendar and select the year
            print("Searching for available days in the calendar...")
            calendar_trigger_element = wait.until(EC.element_to_be_clickable((By.ID, "calendar")))
            driver.execute_script("document.getElementById('calendar').click()")
            print("Clicked on the calendar. The datepicker should now be visible.")

            # Select the year
            year_dropdown = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "ui-datepicker-year")))
            year_dropdown.click()
            select_year = Select(year_dropdown)
            select_year.select_by_visible_text(str(year))
            print(f"Selected year {year}.")

            # Select the month
            month_dropdown = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "ui-datepicker-month")))
            month_dropdown.click()
            select_month = Select(month_dropdown)
            select_month.select_by_index(month_index)
            print(f"Selected month: {month_names[month_index]}.")

            # Find available days
            dias_disponibles = driver.find_elements(
                By.XPATH,
                "//div[@id='ui-datepicker-div']//table[@class='ui-datepicker-calendar']//td[@title='Available']//a"
            )

            if not dias_disponibles:
                print(f"No available days found for {month_names[month_index]} {year}. Skipping...")
                continue

            print(f"Found {len(dias_disponibles)} available days in {month_names[month_index]} {year}.")

            # Initialize a list to store all data for the current month
            datos_cotizaciones = []

            # Loop through all available days in the current month
            for day_index in range(len(dias_disponibles)):
                try:
                    # Reopen the calendar and re-select the year and month
                    calendar_trigger_element = wait.until(EC.element_to_be_clickable((By.ID, "calendar")))
                    driver.execute_script("document.getElementById('calendar').click()")
                    print("Reopened the calendar.")

                    # Select the year
                    year_dropdown = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "ui-datepicker-year")))
                    year_dropdown.click()
                    select_year = Select(year_dropdown)
                    select_year.select_by_visible_text(str(year))
                    print(f"Selected year {year}.")

                    # Select the month
                    month_dropdown = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "ui-datepicker-month")))
                    month_dropdown.click()
                    select_month = Select(month_dropdown)
                    select_month.select_by_index(month_index)
                    print(f"Selected month: {month_names[month_index]}.")

                    # Re-fetch the list of available days
                    dias_disponibles = driver.find_elements(
                        By.XPATH,
                        "//div[@id='ui-datepicker-div']//table[@class='ui-datepicker-calendar']//td[@title='Available']//a"
                    )

                    # Click on the current day
                    day = dias_disponibles[day_index]
                    day.click()
                    print(f"Clicked on day {day_index + 1}.")

                    # Wait for the table to load
                    a = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.table")))

                    fecha_element = driver.find_element(By.XPATH, "//*[@id='dp_cotizacion']")
                    value_date = fecha_element.get_attribute("value")

                    # Parse the current date
                    current_date = datetime.strptime(value_date, "%d/%m/%Y")

                    # Extract data based on the cutoff date
                    if current_date < cutoff_date:
                        # Extract data from the last row
                        a= wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.table")))
                        rows = a.find_elements(By.XPATH, "./tbody/tr")
                        last_row = rows[-1]


                        cells = last_row.find_elements(By.TAG_NAME, "td")
                        if cells:
                            fecha = value_date
                            compra = cells[1].text.strip()
                            venta = cells[2].text.strip()
                            print(f"Extracted data from last row for {fecha}: Compra={compra}, Venta={venta}")
                        else:
                            print("No data found in the last row.")
                        datos_cotizaciones.append(
                            {"Fecha": fecha, "Compra": compra, "Venta": venta}
                        )
                    else:

                        tfoot = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.table tfoot")))
                        rows = tfoot.find_elements(By.XPATH, "./tr")
                        # Extract data from all rows
                        for row in rows:
                            cells = row.find_elements(By.TAG_NAME, "th")
                            if cells:
                                fecha = value_date
                                compra = cells[1].text.strip()
                                venta = cells[2].text.strip()
                                print(f"Extracted data for {fecha}: Compra={compra}, Venta={venta}")

                            datos_cotizaciones.append(
                                {"Fecha": fecha, "Compra": compra, "Venta": venta}
                            )
                except Exception as e:
                    # Handle exception and add a placeholder entry
                    print(f"Exception occurred for day {day_index + 1}: {str(e)}")
                    fecha_element = driver.find_element(By.XPATH, "//*[@id='dp_cotizacion']")
                    value_date = fecha_element.get_attribute("value")
                    datos_cotizaciones.append(
                        {"Fecha": value_date, "Compra": "N/A", "Venta": "N/A"}
                    )
                    continue

            # Save all data for the current month
            if datos_cotizaciones:
                # Create folder if it doesn't exist
                folder_name = "cotizaciones_bcp"
                if not os.path.exists(folder_name):
                    os.makedirs(folder_name)
                    print(f"Created folder: {folder_name}")

                # Create the filename and full file path
                filename = f"cotizacion_{year}_{month_index}_{month_names[month_index]}.csv"
                filepath = os.path.join(folder_name, filename)

                # Save the CSV file
                pd.DataFrame(datos_cotizaciones).to_csv(filepath, index=False)
                print(f"File saved: {filepath}")
            else:
                print(f"No data available to save for {month_names[month_index]} {year}.")

        except Exception as e:
            print(f"Exception occurred for {month_names[month_index]} {year}: {str(e)}")
            continue
driver.quit()

## DESCARGAR TODOS LOS ARCHIVOS DE LA CARPETA COTIZACIONES_BCP


In [ ]:
from google.colab import files

carpeta = "cotizaciones_bcp"

if os.path.exists(carpeta):
    print(f"Comprimiendo la carpeta '{carpeta}' para su descarga...")
    # Comprimir la carpeta
    !zip -r /content/{carpeta}.zip /content/{carpeta}
    print(f"Carpeta '{carpeta}' comprimida exitosamente. Iniciando descarga...")
    # Descargar el archivo ZIP
    files.download(f"{carpeta}.zip")
else:
    print(f"La carpeta '{carpeta}' no existe.")

## AGREGAR TODOS LOS CSV AL DATAFRAME

In [ ]:
import os
import pandas as pd
# Define the folder containing the CSV files
carpeta = "cotizaciones_bcp"

# List all files in the folder and filter for those ending with .csv
archivos_csv = [f for f in os.listdir(carpeta) if f.endswith(".csv")]

# List to store the DataFrames from each file
dataframes = []

print(f"Found {len(archivos_csv)} CSV files in '{carpeta}'. Loading...")

# Read and append each CSV file to the list
for archivo in archivos_csv:
    ruta = os.path.join(carpeta, archivo)
    try:
        # Try reading with UTF-8 encoding first
        df = pd.read_csv(ruta, encoding="utf-8")
        print(f"  ✓ Successfully read {archivo} with utf-8 encoding.")
    except UnicodeDecodeError:
        # If UTF-8 fails, try latin1 encoding
        try:
            df = pd.read_csv(ruta, encoding="latin1")
            print(f"  ✓ Successfully read {archivo} with latin1 encoding.")
        except Exception as e:
            print(f"  ✗ Error reading {archivo} with both utf-8 and latin1: {e}")
            continue # Skip this file if both encodings fail

    # Add the DataFrame to the list
    dataframes.append(df)

# Unir all DataFrames in the list
if dataframes:
    df_total = pd.concat(dataframes, ignore_index=True)
    print(f"\nAll CSV files combined into a single DataFrame with {len(df_total)} rows.")

    # Display the first few rows and information about the combined DataFrame
    print("\n--- First 5 rows of the combined DataFrame ---")
    print(df_total.head())

    print("\n--- Info about the combined DataFrame ---")
    df_total.info()

else:
    print("\nNo CSV files were successfully loaded.")
# Define the name for the output CSV file
output_filename = "combined_cotizaciones.csv"
# Save the combined DataFrame to a CSV file
df_total.to_csv(output_filename, index=False)